In [1]:
%%HTML
<style>
.rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
     font-size: 100%;
}
</style>


In [2]:
import numpy as np

# Metody Numeryczne

## Błędy zaokrągleń

### dr hab. inż. Jerzy Baranowski, Prof. AGH.

## Błędy zaokrągleń
Kolejne nieusuwalne w pełni źródło błędów, nad którym mamy mniejszą kontrolę niż nad błędem metody

## Zaokrąglenie i cyfry znaczące
Liczba $\tilde{y}=\mathrm{rd}(y)$ jest poprawnie zaokrąglona do *d* miejsc po przecinku, jeżeli 

$$
\varepsilon=|y-\tilde{y}|\leq\frac{1}{2}\cdot10^{-d}
$$
*k*-tą cyfrę dziesiętną liczby $\tilde{y}$ nazwiemy znaczącą gdy
$$|y-\tilde{y}|\leq\frac{1}{2}\cdot10^{-k}$$
oraz 
$$|\tilde{y}|\geq10^{-k}
$$

## Rzeczywiste obliczenia zmiennoprzecinkowe
$$
\begin{align}
\mathrm{fl}(x+y)={}&\mathrm{rd}(x+y)\\
\mathrm{fl}(x-y)={}&\mathrm{rd}(x-y)\\
\mathrm{fl}(x\cdot y)={}&\mathrm{rd}(x\cdot y)\\
\mathrm{fl}(x/y)={}&\mathrm{rd}(x/y)\\
\end{align}
$$

## Liczby maszynowe
- Liczba maszynowa, to taka liczba jaką można przedstawić w komputerze. Zbiór tych liczb oznaczamy A
- Dokładność maszynową (epsilon maszynowy) – eps, $\varepsilon_m$,  definiujemy:
$$
\mathrm{eps}=\min\{x\in{A}\colon \mathrm{fl}(1+x)>1,\ x>0\}
$$
Innymi słowy, jest to najmniejsza liczba, którą możemy dodać do 1, aby uzyskać coś większego od 1. 

## Epsilon maszynowy w różnych formatach

Zależy on od liczby bitów na część ułamkową
- Single precision $\varepsilon_m=2^{-24}\approx 5.96\cdot10^{-8}$
- Double precision $\varepsilon_m=2^{-52}\approx 1.11\cdot10^{-16}$

### Przykład

In [3]:
a=10**(-15)
b=10**(-17)
1+a>1,1+b>1

(True, False)

## Maksymalny błąd reprezentacji
Dla każdej liczby rzeczywistej $x$ istnieje taka liczba $\varepsilon$, taka że $|\varepsilon|<\varepsilon_m$, że
$\mathrm{fl}(x)=x(1+\varepsilon)$

Oznacza to, że **błąd względny między liczbą rzeczywistą, a jej najbliższą reprezentacją zmiennoprzecinkową jest zawsze mniejszy od $\varepsilon_m$**

## Przykład 
Rozważmy liczby $x$, $y$ i $z$ takie, że
$$
x+y-z=0
$$
Porównajmy siatkę $[0,4]$ z krokiem $0.01$ co do wartości niespełnienia równania.




*Rysunek dzięki uprzejmości Prof. Danielle Navarro, (Twitter: @djnavarro)*
<img src="img/floating_point_01.png" alt="drawing" width="900"/>


## Lemat Wilkinsona
Błedy zaokrągleń powstałe podczas wykonywania działań zmiennoprzecinkowych są równoważne zastępczemu zaburzeniu liczb, na których wykonujemy działania 

$$
\begin{align}
\mathrm{fl}(x+y)={}&(x+y)(1+\varepsilon_1)\\
\mathrm{fl}(x-y)={}&(x-y)(1+\varepsilon_2)\\
\mathrm{fl}(x\cdot y)={}&(x\cdot y)(1+\varepsilon_3)\\
\mathrm{fl}(x/y)={}&(x/y)(1+\varepsilon_4)\\
|\varepsilon_i|<{}&\varepsilon_m
\end{align}
$$
(dla każdej pary liczb $x,\ y$ zaburzenia zastępcze $\varepsilon_i$ są inne)

## Konsekwencja lematu Wilkinsona
Prawa łączności i rozdzielności operacji matematycznych są ogólnie nieprawdziwe dla obliczeń zmiennoprzecinkowych

### Przykład

In [4]:
a=np.float32(0.23371258*10**(-4))
b=np.float32(0.33678429*10**(2))
c=np.float32(-0.33677811*10**(2))
print([a,b,c])

[2.3371258e-05, 33.67843, -33.67781]


Chcemy obliczyć ``a+b+c``

## Obliczenia

In [5]:
## Podejście 1
d=b+c
wynik_1=a+d
print(wynik_1)

0.0006413522


In [6]:
## Podejście 2
e=a+b
wynik_2=e+c
print(wynik_2)

0.00064086914


## Co tu się porobiło?
![](img/wat.png)

## Konsekwencje obliczen zmiennoprzecinkowych

In [7]:
m_a, e_a = np.frexp(a)
print(m_a,e_a)
m_b,e_b = np.frexp(b)
print(m_b,e_b)
m_c,e_c = np.frexp(c)
print(m_c,e_c)

0.7658294 -15
0.52622545 6
-0.5262158 6


Wykładnik ``a`` od wykładników ``b`` i ``c`` różni się o 21. Oznacza to, że z 23 bitów mantysy liczby ``a`` po sprowadzeniu do wspólnego wykładnika z ``b`` zostaną nam tylko 2 najbardziej znaczące. 

## Konsekwencje cd..
Jeżeli dodajemy małą liczbę do dużej, zawsze musimy się liczyć z zaokrągleniem i to normalne. W tym przypadku jednak dwie duże liczby ``b`` i ``c`` są przeciwnych znaków i bliskie co do wartości bezwzględnej. Wynik tego działania:

In [8]:
m_d,e_d = np.frexp(d)
print(m_d,e_d)
print(wynik_2)

0.6328125 -10
0.00064086914


W konsekwencji dodając ``a`` do ``d`` na zaokrągleniu stracimy jedynie 5 bitów mantysy ``a``.

## O ile się pomyliliśmy (w stosunku do dokładniejszych obliczeń)

In [9]:
a_dbl=(0.23371258*10**(-4))
b_dbl=(0.33678429*10**(2))
c_dbl=(-0.33677811*10**(2))
d_dbl=b_dbl+c_dbl
wynik_dbl=a_dbl+d_dbl
epsilon_1=np.abs((wynik_1)-wynik_dbl)
eta_1=epsilon_1/np.abs(wynik_dbl)
print("Metoda 1: Błąd bezwzględny %10.2e, Błąd względny %10.2e"%(epsilon_1,eta_1))
epsilon_2=np.abs((wynik_2)-wynik_dbl)
eta_2=epsilon_2/np.abs(wynik_dbl)
print("Metoda 2: Błąd bezwzględny %10.2e, Błąd względny %10.2e"%(epsilon_2,eta_2))



Metoda 1: Błąd bezwzględny   1.91e-08, Błąd względny   2.97e-05
Metoda 2: Błąd bezwzględny   5.02e-07, Błąd względny   7.83e-04


# Przenoszenie się błędów zaokrągleń
Korzystając z rachunku różniczkowego (różniczkowa analiza błędów) możemy podać wzór na przenoszenie się błędów.

Niech $y=\varphi(x_1,\ x_2,,\ldots\ x_n)$ będzie wielkością, którą chcemy obliczyć a $x_i$ są zaokrąglone z błędem $\varepsilon_{x_i}$. Błąd względny wyliczania $y$ wynosi w przybliżeniu:

$$
\varepsilon_y = \sum_{i=0}^n \frac{x_i}{\varphi(\mathbf{x})}
\cdot \frac{\partial\varphi(\mathbf{x})}{\partial x_i}\cdot\varepsilon_{x_i}
$$



# Nieunikniony błąd obliczeń
Ze względu na zaokrąglenia pewnych błędów nigdy nie unikniemy. Nieunikniony błąd wartości składa się z błędu wyliczenia wartości (przeniesienia błędów) oraz samego błędu zaokrąglenia:

$$
\frac{\Delta y}{y} = \epsilon_y + \mathrm{eps}
$$

## Przykład
Wyliczanie pierwiastka równania kwadratowego $$y^2+2py-q=0$$ o mniejszej wartości bezwzględnej:
$$ y=-p+\sqrt{p^2+q} $$
można policzyć, że 
$$
\varepsilon_y=-\frac{p}{\sqrt{p^2+q}}\varepsilon_p+\frac{p+\sqrt{p^2+q}}{2\sqrt{p^2+q}}\varepsilon_q
$$

## Analiza błędu nieuniknionego
Ponieważ dla $q>0$ mamy

$$
\left|\frac{p}{\sqrt{p^2+q}}\right|\leq1,\quad \left|\frac{p+\sqrt{p^2+q}}{2\sqrt{p^2+q}}\right|\leq1
$$
to wtedy mamy (przyjmując, że nie zachodzi $p^2\approx -q$)
$$
\mathrm{eps}\leq\left|\frac{\Delta y}{y}\right| = |\epsilon_y + \mathrm{eps}|\leq 3 \mathrm{eps}
$$

## Porównanie algorytmów
Rozpartrzmy dwa sposoby wyliczania $y$ dla $p$ i $q$ mniejszych od zera

$$
\begin{aligned}
s:={}&p^2\\
t:={}&s+q\\
u:={}&\sqrt{t}\\
y:={}&-p+u
\end{aligned}
\quad \quad \quad \quad
\begin{aligned}
s:={}&p^2\\
t:={}&s+q\\
u:={}&\sqrt{t}\\
v:={}&p+u\\
y:={}&q/v
\end{aligned}
$$


## Algorytm 1
Podstawowym źródłem błędu będzie wzmocnienie błędu zaokrąglenia wyliczania pierwiastka z $t$ poprzez odejmowanie dwóch liczb przy wyliczaniu $y$
$$\varepsilon_y=\frac{p\sqrt{p^2+q}+p^2+q}{q}\varepsilon=\kappa\varepsilon$$
$\kappa$ można oszacować z dołu, przez 
$$
\kappa>\frac{2 p^2}{q} >0
$$
co oznacza, że dla małych $q$ błąd obliczeń będzie dużo większy niż błąd nieunikniony.


## Algorytm 2
W tym algorytmie zakokrąglenie przez odejmowanie nie wystąpi

$$
\varepsilon_y = -\frac{\sqrt{p^2+q}}{p+\sqrt{p^2+q}}\varepsilon = \kappa\varepsilon
$$
w tym przypadku zawsze $|\kappa|<1$.

In [10]:
def algorytm_1(p,q):
    s=p**2
    t=s+q
    u=np.sqrt(t)
    return u-p

def algorytm_2(p,q):
    s=p**2
    t=s+q
    u=np.sqrt(t)
    v=p+u
    return q/v

# Porównanie obliczeń

In [11]:
p=1000
q=0.018000000081
exact_sol=np.max(np.roots([1,2*p,-q]))

In [12]:
epsilon_1=np.abs((algorytm_1(p,q))-exact_sol)
eta_1=epsilon_1/np.abs(exact_sol)
epsilon_2=np.abs((algorytm_2(p,q))-exact_sol)
eta_2=epsilon_2/np.abs(exact_sol)

In [13]:
print('Algorytm 1')
print(algorytm_1(p,q))
print('Algorytm 2')
print(algorytm_2(p,q))
print('Rozwiązanie dokładne')
print(exact_sol)
print("Algorytm 1: Błąd bezwzględny %10.2e, Błąd względny %10.2e"%(epsilon_1,eta_1))
print("Algorytm 2: Błąd bezwzględny %10.2e, Błąd względny %10.2e"%(epsilon_2,eta_2))

Algorytm 1
8.999999977277184e-06
Algorytm 2
9e-06
Rozwiązanie dokładne
9e-06
Algorytm 1: Błąd bezwzględny   2.27e-14, Błąd względny   2.52e-09
Algorytm 2: Błąd bezwzględny   0.00e+00, Błąd względny   0.00e+00


# Metody Numeryczne

## Ocena algorytmów numerycznych

### dr hab. inż. Jerzy Baranowski, Prof. AGH.

## Notacja O duże
- Mówimy, że dla wielkości zależnej od parametru np. $F(n)$ zachodzi
$$ 
F(n)=O(G(n))
$$
jeżeli istnieje taka stała $C$, że przy $n$ zmierzającym do nieskończoności (odpowiednio dużym), mamy
$$F(n)≤C G(n)$$
- Jeżeli interesuje nas $O(c)$, gdzie $c$ jest stałą, zależność ta ma zachodzić niezależnie od wielkości parametru.
- Mówimy potocznie, gdy błąd jest równy $O(n^2)$, że błąd jest rzędu $n^2$
  

## Ocena algorytmu
- Naszym celem jest obliczenie pewnej wielkości $f(x)$, zależnej od danych wejściowych $x$
- W przypadku obliczeń komputerowych zawsze mamy do czynienia z obliczaniem przybliżonym stąd algorytm obliczania $f(x)$ będziemy oznaczać jako $f^*(x)$
- Dane w komputerze również są reprezentowane w sposób zaokrąglony, więc będziemy je oznaczać jako $x^*$

## Uwarunkowanie problemu

- Mówimy, że problem $f(x)$ jest dobrze uwarunkowany, jeżeli mała zmiana $x$ powoduje małą zmianę w $f(x)$
- Problem jest źle uwarunkowany, jeżeli mała zmiana $x$ powoduje dużą zmianę w $f(x)$
- Miarą uwarunkowania jest stała $\kappa$ (kappa), która (nieformalnie) określa największy iloraz zaburzeń $f(x)$  wywołanych przez najmniejsze zaburzenia $x$.
- Stałą $\kappa$ można wyliczyć tylko w niektórych probemach

## Dokładność algorytmu
- Algorytm jest dokładny, jeżeli
$$
\frac{\Vert f^*(x)-f(x) \Vert}{\Vert f(x)\Vert}=O(\varepsilon_m)
$$
- Zagwarantowanie, że algorytm jest dokładny wg tej definicji jest niezwykle trudne, zwłaszcza dla źle uwarunkowanych problemów

## Stabilność algorytmu

Mówimy, że algorytm jest stabilny, gdy dla każdego $x$, zachodzi
$$
\frac{\Vert f^*(x)-f(x^*) \Vert}{\Vert f(x^*)\Vert}=O(\varepsilon_m)
$$
dla takich $x^*$, że
$$\frac{\Vert x-x^* \Vert}{\Vert x\Vert}=O(\varepsilon_m)$$
Innymi słowy
**Stabilny algorytm daje prawie dobrą odpowiedź na prawie dobre pytanie**

## Stabilność wsteczna algorytmu
Algorytm jest stabilny wstecznie, jeżeli dla każdego $x$, zachodzi
$$f^*(x)=f(x^*)$$
dla takich $x^*$, że
$$\frac{\Vert x-x^* \Vert}{\Vert x\Vert}=O(\varepsilon_m)$$
Innymi słowy
**Stabilny wstecznie algorytm daje prawidłową odpowiedź na prawie dobre pytanie**





## Dokładność algorytmów stabilnych wstecznie przy złym uwarunkowaniu
Jeśli algorytm jest stabilny wstecznie, to jego błąd względny pogarsza się proporcjonalnie do stałej uwarunkowania tj. $O(\kappa\varepsilon_m)$

## Złożoność obliczeniowa algorytmu
- O złożoności obliczeniowej mówimy jako o liczbie zasobów potrzebnych do wyliczenia algorytmu.
- Złożoność określamy zazwyczaj jako funkcję pewnych wielkości wejściowych np. liczby zmiennych lub liczby ograniczeń
- Istnieją różne rodzaje


## Złożoność arytmetyczna
Liczba operacji arytmetycznych (zazwyczaj mnożeń) potrzebnych do rozwiązania problemu. Zazwyczaj dotyczy to problemów algebry numerycznej gdzie jesteśmy w stanie oszacować liczbę działań.

Przykładowo rozwiązanie układu równań liniowych za pomocą metody eliminacji Gaussa to $O(2/3 n^3)$ gdzie $n$ to liczba zmiennych

## Złożoność czasowa
Ile potrzeba czasu na wyliczenie algorytmu. Zazwyczaj przy założeniu stałego czasu obliczeń równoważna złożoności arytmetycznej. Przy bardziej skomplikowanych problemach (gdzie pojawia się zarządzanie pamięcią) może się różnić. 

Nie używa się czasu wprost.

## Złożoność pamięciowa 
Ilość pamięci potrzebnej do realizacji obliczeń

## Bardziej złożone problemy

Przy rozwiązywaniu równań różniczkowych lub optymalizacji za miarę złożoności określa się liczbę wywołań prawej strony równania różniczkowego lub funkcji celu.